## Import Library

In [1]:
%tensorflow_version 2.x
import json
import requests
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import mean_absolute_error , r2_score , mean_squared_error
from sklearn.model_selection import GridSearchCV
%matplotlib inline

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


## Create dataset

In [2]:
endpoint = 'https://min-api.cryptocompare.com/data/histoday'
res = requests.get(endpoint + '?fsym=BTC&tsym=CAD&limit=1000')
hist = pd.DataFrame(json.loads(res.content)['Data'])
hist = hist.set_index('time')
hist.index = pd.to_datetime(hist.index, unit='s')

In [ ]:
hist

,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol
time,,,,,,,,
2020-02-04,12299.80,12000.00,12234.30,29.93,363396.83,12164.70,direct,
2020-02-05,12762.60,12110.20,12164.70,85.37,1068653.94,12635.70,direct,
2020-02-06,12935.00,12477.40,12635.70,62.95,802245.42,12912.20,direct,
2020-02-07,13000.00,12842.70,12912.20,54.37,703314.44,12994.50,direct,
2020-02-08,13088.10,12759.60,12994.50,22.38,289608.42,13085.90,direct,
...,...,...,...,...,...,...,...,...
2022-10-27,28240.26,27423.92,28129.98,105.92,2958314.22,27529.62,direct,
2022-10-28,28249.54,27268.07,27529.62,81.60,2263533.08,28091.08,direct,
2022-10-29,28643.57,28026.38,28091.08,111.80,3168835.79,28327.15,direct,


In [3]:
hist.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1001 entries, 2020-02-19 to 2022-11-15
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   high              1001 non-null   float64
 1   low               1001 non-null   float64
 2   open              1001 non-null   float64
 3   volumefrom        1001 non-null   float64
 4   volumeto          1001 non-null   float64
 5   close             1001 non-null   float64
 6   conversionType    1001 non-null   object 
 7   conversionSymbol  1001 non-null   object 
dtypes: float64(6), object(2)
memory usage: 70.4+ KB


## Data Preprocessing

Remove Unwanted coloums

In [4]:
hist.drop(["conversionType", "conversionSymbol"], axis = 'columns', inplace = True)

check null values

In [5]:
hist.isnull().sum()

high          0
low           0
open          0
volumefrom    0
volumeto      0
close         0
dtype: int64

Normalize The Dataset

In [6]:
from sklearn.preprocessing import MinMaxScaler
hist_columns = hist.columns
scaler = MinMaxScaler()
hist = scaler.fit_transform(hist)
hist = pd.DataFrame(hist)
hist.columns = hist_columns

Describe the Dataset

In [7]:
hist.describe()

,high,low,open,volumefrom,volumeto,close
count,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000,1001.000000
mean,0.412818,0.416746,0.414608,0.063341,0.076320,0.414715
std,0.274984,0.264350,0.272952,0.122727,0.146978,0.272829
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.128813,0.149060,0.134329,0.003096,0.001989,0.135242
50%,0.413179,0.412352,0.415406,0.005440,0.003712,0.415406
75%,0.638019,0.636080,0.636631,0.017527,0.013971,0.636631
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Divided the Dataset

In [8]:
X = hist.drop(columns = "close")
y = hist.close
print(X)
print(y)

          high       low      open  volumefrom  volumeto
0     0.078469  0.089767  0.086043    0.005805  0.001636
1     0.069948  0.091426  0.076662    0.002381  0.000561
2     0.072518  0.093835  0.077077    0.002551  0.000623
3     0.070364  0.094276  0.078165    0.000445  0.000000
4     0.074626  0.094997  0.077618    0.001198  0.000231
...        ...       ...       ...         ...       ...
996   0.206387  0.211429  0.215571    0.007305  0.003775
997   0.193845  0.210670  0.204863    0.006213  0.003112
998   0.187130  0.199389  0.193818    0.005267  0.002565
999   0.195388  0.201231  0.191718    0.006347  0.003157
1000  0.191796  0.215130  0.198927    0.001528  0.000748

[1001 rows x 5 columns]
0       0.076662
1       0.077077
2       0.078165
3       0.077618
4       0.082977
          ...   
996     0.204863
997     0.193818
998     0.191718
999     0.198927
1000    0.202047
Name: close, Length: 1001, dtype: float64


Train_test_split

In [9]:
from sklearn.model_selection import train_test_split
test_size = eval(input("test size= "))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test_size, random_state=42)

test size= 0.4


# Using Random forest Regressor Model

In [10]:
from sklearn.ensemble import RandomForestRegressor
RF = RandomForestRegressor()
RF.fit(X_train,y_train)

RandomForestRegressor()

predict from model

In [11]:
rf_y_pred = RF.predict(X_test)
rf_y_pred

array([0.46194242, 0.25637845, 0.64021369, 0.27383493, 0.85170561,
       0.68879523, 0.75012046, 0.42905574, 0.658755  , 0.07328275,
       0.27008186, 0.07595649, 0.55561398, 0.53534652, 0.25478761,
       0.30227897, 0.23838581, 0.30235127, 0.64383755, 0.43582569,
       0.72751158, 0.10985103, 0.71580893, 0.57980365, 0.09684857,
       0.10482863, 0.07989304, 0.27369991, 0.30207721, 0.65441803,
       0.47564208, 0.07667553, 0.9358529 , 0.45163674, 0.74030974,
       0.09003176, 0.85564128, 0.59653814, 0.25148355, 0.03984437,
       0.50317755, 0.07417141, 0.35676164, 0.250071  , 0.22866435,
       0.2224497 , 0.25323961, 0.81933444, 0.83913271, 0.26348914,
       0.60476762, 0.26742998, 0.1760128 , 0.75846086, 0.00629542,
       0.02664769, 0.90081431, 0.06366079, 0.09660703, 0.30498993,
       0.21507587, 0.03223876, 0.67352937, 0.09681484, 0.90327368,
       0.536764  , 0.39815847, 0.25405444, 0.05803916, 0.653254  ,
       0.08562188, 0.41393509, 0.69672416, 0.3983599 , 0.23339

##Evalute the Model


1.   Mean Absolute error
2.   Mean Squared error
3.   r2 score

In [ ]:
MAE = mean_absolute_error(rf_y_pred, y_test)
MSE=mean_squared_error(rf_y_pred, y_test)
R2=r2_score(rf_y_pred, y_test)
print("mean absolute error value is = ",MAE)
print("mean squared error value is = ",MSE)
print("R2 score value is = ",R2)

mean absolute error value is =  0.007213494848629944
mean squared error value is =  0.0001244783578933231
R2 score value is =  0.9985874638969592


#GridSearchcv of Rfregressor

In [ ]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

In [ ]:
RF_grid_search = GridSearchCV(estimator = RF, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)

with train part of dataset

In [ ]:
RF_grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 288 candidates, totalling 864 fits


GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [80, 90, 100, 110],
                         'max_features': [2, 3], 'min_samples_leaf': [3, 4, 5],
                         'min_samples_split': [8, 10, 12],
                         'n_estimators': [100, 200, 300, 1000]},
             verbose=2)

In [ ]:
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",RF_grid_search.best_estimator_)
print("\n The best score across ALL searched params:\n",RF_grid_search.best_score_)
print("\n The best parameters across ALL searched params:\n",RF_grid_search.best_params_)

 Results from Grid Search 

 The best estimator across ALL searched params:
 RandomForestRegressor(max_depth=90, max_features=3, min_samples_leaf=3,
                      min_samples_split=8, n_estimators=300)

 The best score across ALL searched params:
 0.9981244787253604

 The best parameters across ALL searched params:
 {'bootstrap': True, 'max_depth': 90, 'max_features': 3, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 300}


With test part of Dataset

In [ ]:
RF_grid_search.fit(X_test, y_test)

Fitting 3 folds for each of 288 candidates, totalling 864 fits


GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [80, 90, 100, 110],
                         'max_features': [2, 3], 'min_samples_leaf': [3, 4, 5],
                         'min_samples_split': [8, 10, 12],
                         'n_estimators': [100, 200, 300, 1000]},
             verbose=2)

In [ ]:
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",RF_grid_search.best_estimator_)
print("\n The best score across ALL searched params:\n",RF_grid_search.best_score_)
print("\n The best parameters across ALL searched params:\n",RF_grid_search.best_params_)

 Results from Grid Search 

 The best estimator across ALL searched params:
 RandomForestRegressor(max_depth=100, max_features=3, min_samples_leaf=3,
                      min_samples_split=8)

 The best score across ALL searched params:
 0.997140403992587

 The best parameters across ALL searched params:
 {'bootstrap': True, 'max_depth': 100, 'max_features': 3, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 100}


#Using Linear regression Lasso Part

In [ ]:
from sklearn import linear_model
LR = linear_model.Lasso()
LR.fit(X_train,y_train)

Lasso()

Predict from the Model

In [ ]:
LR_y_Pred = LR.predict(X_test)
LR_y_Pred

array([0.39529806, 0.39529806, 0.39529806, 0.39529806, 0.39529806,
       0.39529806, 0.39529806, 0.39529806, 0.39529806, 0.39529806,
       0.39529806, 0.39529806, 0.39529806, 0.39529806, 0.39529806,
       0.39529806, 0.39529806, 0.39529806, 0.39529806, 0.39529806,
       0.39529806, 0.39529806, 0.39529806, 0.39529806, 0.39529806,
       0.39529806, 0.39529806, 0.39529806, 0.39529806, 0.39529806,
       0.39529806, 0.39529806, 0.39529806, 0.39529806, 0.39529806,
       0.39529806, 0.39529806, 0.39529806, 0.39529806, 0.39529806,
       0.39529806, 0.39529806, 0.39529806, 0.39529806, 0.39529806,
       0.39529806, 0.39529806, 0.39529806, 0.39529806, 0.39529806,
       0.39529806, 0.39529806, 0.39529806, 0.39529806, 0.39529806,
       0.39529806, 0.39529806, 0.39529806, 0.39529806, 0.39529806,
       0.39529806, 0.39529806, 0.39529806, 0.39529806, 0.39529806,
       0.39529806, 0.39529806, 0.39529806, 0.39529806, 0.39529806,
       0.39529806, 0.39529806, 0.39529806, 0.39529806, 0.39529

##Evalute the Model


1.   Mean Absolute error
2.   Mean Squared error
3.   r2 score

In [ ]:
MAE = mean_absolute_error(LR_y_Pred, y_test)
MSE=mean_squared_error(LR_y_Pred, y_test)
R2=r2_score(LR_y_Pred, y_test)
print("mean absolute error value is = ",MAE)
print("mean squared error value is = ",MSE)
print("R2 score value is = ",R2)

mean absolute error value is =  0.27149388601393387
mean squared error value is =  0.08899932101314266
R2 score value is =  -2.888193093176708e+31


# Gridsearch of Lasso

In [ ]:
parameters = {
    "alpha" : [0,0.1,0.01,0.05,0.5,1],
    "normalize" : [True, False],
    "selection" : ["cyclic", "random"]
    
}

In [ ]:
LR_grid_search = GridSearchCV(LR, parameters, cv = 5)

with train part of dataset

In [ ]:
LR_grid_search.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:680: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn

GridSearchCV(cv=5, estimator=Lasso(),
             param_grid={'alpha': [0, 0.1, 0.01, 0.05, 0.5, 1],
                         'normalize': [True, False],
                         'selection': ['cyclic', 'random']})

In [ ]:
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",LR_grid_search.best_estimator_)
print("\n The best score across ALL searched params:\n",LR_grid_search.best_score_)
print("\n The best parameters across ALL searched params:\n",LR_grid_search.best_params_)

 Results from Grid Search 

 The best estimator across ALL searched params:
 Lasso(alpha=0, normalize=True)

 The best score across ALL searched params:
 0.9989083242326222

 The best parameters across ALL searched params:
 {'alpha': 0, 'normalize': True, 'selection': 'cyclic'}


In [ ]:
LR_grid_search.fit(X_test, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:680: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn

GridSearchCV(cv=5, estimator=Lasso(),
             param_grid={'alpha': [0, 0.1, 0.01, 0.05, 0.5, 1],
                         'normalize': [True, False],
                         'selection': ['cyclic', 'random']})

In [ ]:
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",LR_grid_search.best_estimator_)
print("\n The best score across ALL searched params:\n",LR_grid_search.best_score_)
print("\n The best parameters across ALL searched params:\n",LR_grid_search.best_params_)

 Results from Grid Search 

 The best estimator across ALL searched params:
 Lasso(alpha=0, normalize=True)

 The best score across ALL searched params:
 0.9990399738094506

 The best parameters across ALL searched params:
 {'alpha': 0, 'normalize': True, 'selection': 'cyclic'}


#Using Gradientboost Algorithm

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
GB = GradientBoostingRegressor()
GB.fit(X_train, y_train)

GradientBoostingRegressor()

predict the values

In [ ]:
GB_y_pred = GB.predict(X_test)
GB_y_pred

array([0.83987579, 0.45089069, 0.9515632 , 0.25907537, 0.23450454,
       0.71666034, 0.66901579, 0.7475785 , 0.85330052, 0.08248817,
       0.58975217, 0.03578433, 0.93794473, 0.75788472, 0.49271482,
       0.67142835, 0.07373334, 0.55185235, 0.82126517, 0.11107752,
       0.88245757, 0.03823898, 0.10806232, 0.85070253, 0.09607396,
       0.08625846, 0.05705371, 0.26751685, 0.6408381 , 0.92308563,
       0.10056892, 0.06389244, 0.4422714 , 0.85364307, 0.10918448,
       0.04801879, 0.46595751, 0.60103569, 0.55591528, 0.04254551,
       0.10399153, 0.0578033 , 0.11966794, 0.36211052, 0.11309792,
       0.07530285, 0.55998713, 0.51340921, 0.3123229 , 0.39976313,
       0.72837137, 0.59490502, 0.07735006, 0.55402112, 0.069897  ,
       0.06417247, 0.47961603, 0.04801879, 0.08873726, 0.59898492,
       0.11349038, 0.03823898, 0.85135967, 0.0877811 , 0.45410931,
       0.86935594, 0.5023527 , 0.52727027, 0.03578433, 0.8515265 ,
       0.05373076, 0.69488188, 0.50872049, 0.58187656, 0.10806

##Evalute the Model


1.   Mean Absolute error
2.   Mean Squared error
3.   r2 score

In [ ]:
MAE = mean_absolute_error(GB_y_pred, y_test)
MSE=mean_squared_error(GB_y_pred, y_test)
R2=r2_score(GB_y_pred, y_test)
print("mean absolute error value is = ",MAE)
print("mean squared error value is = ",MSE)
print("R2 score value is = ",R2)

mean absolute error value is =  0.00783163138563432
mean squared error value is =  0.00013900348501789565
R2 score value is =  0.9984211551860152


#Gridsearch of Gradient Boost

In [ ]:
parameters = {'learning_rate': [0.01,0.02,0.03,0.04],
                  'subsample'    : [0.9, 0.5, 0.2, 0.1],
                  'n_estimators' : [100,500,1000, 1500],
                  'max_depth'    : [4,6,8,10]
                 }

In [ ]:
GB_grid_search = GridSearchCV(estimator=GB, param_grid = parameters, cv = 3, n_jobs=-1)

with train part of dataset

In [ ]:
GB_grid_search.fit(X_train,y_train)

GridSearchCV(cv=3, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.02, 0.03, 0.04],
                         'max_depth': [4, 6, 8, 10],
                         'n_estimators': [100, 500, 1000, 1500],
                         'subsample': [0.9, 0.5, 0.2, 0.1]})

In [ ]:
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",GB_grid_search.best_estimator_)
print("\n The best score across ALL searched params:\n",GB_grid_search.best_score_)
print("\n The best parameters across ALL searched params:\n",GB_grid_search.best_params_)

With the test part 

In [ ]:
GB_grid_search.fit(X_test,y_test)

GridSearchCV(cv=3, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.02, 0.03, 0.04],
                         'max_depth': [4, 6, 8, 10],
                         'n_estimators': [100, 500, 1000, 1500],
                         'subsample': [0.9, 0.5, 0.2, 0.1]})

In [ ]:
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",GB_grid_search.best_estimator_)
print("\n The best score across ALL searched params:\n",GB_grid_search.best_score_)
print("\n The best parameters across ALL searched params:\n",GB_grid_search.best_params_)

 Results from Grid Search 

 The best estimator across ALL searched params:
 GradientBoostingRegressor(learning_rate=0.02, max_depth=8, n_estimators=1000,
                          subsample=0.5)

 The best score across ALL searched params:
 0.9977618302279385

 The best parameters across ALL searched params:
 {'learning_rate': 0.02, 'max_depth': 8, 'n_estimators': 1000, 'subsample': 0.5}


#Using LSTM model

In [12]:
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, LSTM

In [13]:
# The LSTM architecture
regressor = Sequential()
# First LSTM layer with Dropout regularisation
regressor.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],1)))
regressor.add(Dropout(0.2))
# Second LSTM layer
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))
# Third LSTM layer
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))
# Fourth LSTM layer
regressor.add(LSTM(units=50))
regressor.add(Dropout(0.2))
# The output layer
regressor.add(Dense(units=1))

# Compiling the RNN
regressor.compile(optimizer='adam',loss='mean_squared_error')
# Fitting to the training set
regressor.fit(X_train,y_train,epochs=30,batch_size=32)

Epoch 1/30
19/19 [==============================] - 6s 13ms/step - loss: 0.1360
Epoch 2/30
19/19 [==============================] - 0s 12ms/step - loss: 0.0381
Epoch 3/30
19/19 [==============================] - 0s 12ms/step - loss: 0.0122
Epoch 4/30
19/19 [==============================] - 0s 12ms/step - loss: 0.0055
Epoch 5/30
19/19 [==============================] - 0s 13ms/step - loss: 0.0050
Epoch 6/30
19/19 [==============================] - 0s 12ms/step - loss: 0.0040
Epoch 7/30
19/19 [==============================] - 0s 13ms/step - loss: 0.0039
Epoch 8/30
19/19 [==============================] - 0s 12ms/step - loss: 0.0036
Epoch 9/30
19/19 [==============================] - 0s 11ms/step - loss: 0.0036
Epoch 10/30
19/19 [==============================] - 0s 12ms/step - loss: 0.0032
Epoch 11/30
19/19 [==============================] - 0s 13ms/step - loss: 0.0029
Epoch 12/30
19/19 [==============================] - 0s 12ms/step - loss: 0.0030
Epoch 13/30
19/19 [==================

predict the values

In [14]:
LSTM_y_pred = regressor.predict(X_test)
LSTM_y_pred

13/13 [==============================] - 3s 7ms/step


array([[0.5047215 ],
       [0.263136  ],
       [0.65032053],
       [0.28190738],
       [0.86534375],
       [0.73001903],
       [0.77308947],
       [0.47881988],
       [0.681001  ],
       [0.08362322],
       [0.28887305],
       [0.0875506 ],
       [0.56698143],
       [0.5629197 ],
       [0.26229483],
       [0.30853093],
       [0.24355632],
       [0.30955988],
       [0.6972864 ],
       [0.4339044 ],
       [0.7620395 ],
       [0.12160921],
       [0.73589206],
       [0.6205299 ],
       [0.10918671],
       [0.11462279],
       [0.09296147],
       [0.2818298 ],
       [0.31305674],
       [0.6668853 ],
       [0.49300003],
       [0.08786376],
       [0.92862797],
       [0.492146  ],
       [0.76259583],
       [0.10342421],
       [0.8740099 ],
       [0.65591764],
       [0.25461802],
       [0.05524725],
       [0.5085626 ],
       [0.08427741],
       [0.37717575],
       [0.25850302],
       [0.23795213],
       [0.232187  ],
       [0.26030436],
       [0.854

##Evalute the Model


1.   Mean Absolute error
2.   Mean Squared error
3.   r2 score

In [15]:
MAE = mean_absolute_error(LSTM_y_pred, y_test)
MSE=mean_squared_error(LSTM_y_pred, y_test)
R2=r2_score(LSTM_y_pred, y_test)
print("mean absolute error value is = ",MAE)
print("mean squared error value is = ",MSE)
print("R2 score value is = ",R2)

mean absolute error value is =  0.01822307294246395
mean squared error value is =  0.000576345052974757
R2 score value is =  0.992302081335751


#Gridsearch with LSTM

In [16]:
from keras.wrappers.scikit_learn import KerasClassifier

In [17]:
parameters = {    'batch_size' : [20,32,40],
              'epochs' : [8,30,50],
              'optimizer' : ['adam','Adadelta','rmsprop']
                 }

In [18]:
def build_lstm(optimizer):
    regressor = Sequential()
    # First LSTM layer with Dropout regularisation
    regressor.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],1)))
    regressor.add(Dropout(0.2))
    # Second LSTM layer
    regressor.add(LSTM(units=50, return_sequences=True))
    regressor.add(Dropout(0.2))
    # Third LSTM layer
    regressor.add(LSTM(units=50, return_sequences=True))
    regressor.add(Dropout(0.2))
    # Fourth LSTM layer
    regressor.add(LSTM(units=50))
    regressor.add(Dropout(0.2))
    # The output layer
    regressor.add(Dense(units=1))
    # Compiling the RNN
    regressor.compile(loss='mean_squared_error', optimizer = optimizer,metrics=["accuracy"])
    return regressor

In [19]:
grid_model = KerasClassifier(build_fn=build_lstm)
LSTM_grid_search  = GridSearchCV(estimator = grid_model,
                            param_grid = parameters,
                            cv = 3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """Entry point for launching an IPython kernel.


With the train part of dataset

In [20]:
LSTM_grid_search.fit(X_train, y_train)

Epoch 1/8
20/20 [==============================] - 5s 9ms/step - loss: 53009.3906 - accuracy: 0.0025
Epoch 2/8
20/20 [==============================] - 0s 9ms/step - loss: 51150.0586 - accuracy: 0.0025
Epoch 3/8
20/20 [==============================] - 0s 9ms/step - loss: 49413.5000 - accuracy: 0.0025
Epoch 4/8
20/20 [==============================] - 0s 9ms/step - loss: 48850.8984 - accuracy: 0.0025
Epoch 5/8
20/20 [==============================] - 0s 10ms/step - loss: 48368.5352 - accuracy: 0.0025
Epoch 6/8
20/20 [==============================] - 0s 8ms/step - loss: 47909.8594 - accuracy: 0.0025
Epoch 7/8
20/20 [==============================] - 0s 9ms/step - loss: 47438.7734 - accuracy: 0.0025
Epoch 8/8
10/10 [==============================] - 2s 3ms/step - loss: 48974.6133 - accuracy: 0.0000e+00
Epoch 1/8
20/20 [==============================] - 5s 8ms/step - loss: 53009.6406 - accuracy: 0.0050
Epoch 2/8
20/20 [==============================] - 0s 9ms/step - loss: 50906.4141 - ac

GridSearchCV(cv=3,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f11e2e0c990>,
             param_grid={'batch_size': [20, 32, 40], 'epochs': [8, 30, 50],
                         'optimizer': ['adam', 'Adadelta', 'rmsprop']})

In [ ]:
print(" Results from Grid Search " )
# print("\n The best estimator across ALL searched params:\n",grid_search.best_estimator_)
print("\n The best score across ALL searched params:\n",LSTM_grid_search.best_score_)
print("\n The best parameters across ALL searched params:\n",LSTM_grid_search.best_params_)

 Results from Grid Search 

 The best score across ALL searched params:
 0.0016666666294137638

 The best parameters across ALL searched params:
 {'batch_size': 20, 'epochs': 8, 'optimizer': 'adam'}


with the test part of data set

In [ ]:
LSTM_grid_search.fit(X_test, y_test)

Epoch 1/8
14/14 [==============================] - 7s 12ms/step - loss: 23598.7930 - accuracy: 0.0037
Epoch 2/8
14/14 [==============================] - 0s 13ms/step - loss: 23231.5098 - accuracy: 0.0075
Epoch 3/8
14/14 [==============================] - 0s 14ms/step - loss: 21645.0664 - accuracy: 0.0037
Epoch 4/8
14/14 [==============================] - 0s 12ms/step - loss: 20874.0645 - accuracy: 0.0037
Epoch 5/8
14/14 [==============================] - 0s 14ms/step - loss: 20609.2617 - accuracy: 0.0037
Epoch 6/8
14/14 [==============================] - 0s 13ms/step - loss: 20395.9551 - accuracy: 0.0037
Epoch 7/8
14/14 [==============================] - 0s 13ms/step - loss: 20176.2461 - accuracy: 0.0037
Epoch 8/8
7/7 [==============================] - 2s 4ms/step - loss: 22401.0254 - accuracy: 0.0000e+00
Epoch 1/8
14/14 [==============================] - 8s 12ms/step - loss: 23608.2715 - accuracy: 0.0037
Epoch 2/8
14/14 [==============================] - 0s 12ms/step - loss: 23339.357

GridSearchCV(cv=3,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7faf95ee0690>,
             param_grid={'batch_size': [20, 32, 40], 'epochs': [8, 30, 50],
                         'optimizer': ['adam', 'Adadelta', 'rmsprop']})

In [ ]:
print(" Results from Grid Search " )
# print("\n The best estimator across ALL searched params:\n",grid_search.best_estimator_)
print("\n The best score across ALL searched params:\n",LSTM_grid_search.best_score_)
print("\n The best parameters across ALL searched params:\n",LSTM_grid_search.best_params_)

 Results from Grid Search 

 The best score across ALL searched params:
 0.0025062657271822295

 The best parameters across ALL searched params:
 {'batch_size': 20, 'epochs': 8, 'optimizer': 'Adadelta'}
